# **Admission Prediction Using PySpark**

201-15-14207 --- *Tanjim Khan Nokib*


                                              ---  Admission Prediction Using PySpark   ---- Admission Prediction DataSet




201-15-13934 --- *Zayed Bin Zamal*




In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=3a0c4ca50ac5f07967458e449cfe9f012c44abdd50dc6d57c482f50fd4c8943c
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

spark= SparkSession.builder.appName("Admission Prediction Using PySpark By Tanjim Khan Nokib & Zayed Bin Zamal.").getOrCreate()

In [5]:
spark

In [6]:
! git clone https://github.com/tanjimkhannokib/Bigdata--201-15-14207

Cloning into 'Bigdata--201-15-14207'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 32.63 KiB | 2.17 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [7]:
df = spark.read.csv('/content/Bigdata--201-15-14207/Dataset/Admission_Prediction.csv', header=True, inferSchema=True)

In [8]:
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [9]:
shape = (df.count(), len(df.columns))

print(shape)

(500, 9)


In [10]:
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [11]:
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

In [12]:
df= df.drop('Serial No')

In [13]:
df.show(2)

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
+---------+-----------+-----------------+---+---+----+--------+---------------+
only showing top 2 rows



In [14]:
print(df[df['GRE Score'].isNull()].count())

0


In [15]:
for col in df.columns:
  print(col + ':' , df[df[col].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


### Correlation Analysis & Feature Selection

In [16]:
print(df.stat.corr('GRE Score', 'Chance of Admit'))

0.8103506354632601


In [17]:
for col in df.columns:
  print(f"{col} is {round(df.stat.corr(col, 'Chance of Admit'),3)} correlated with the target variable Chance of Admit")

GRE Score is 0.81 correlated with the target variable Chance of Admit
TOEFL Score is 0.792 correlated with the target variable Chance of Admit
University Rating is 0.69 correlated with the target variable Chance of Admit
SOP is 0.684 correlated with the target variable Chance of Admit
LOR is 0.645 correlated with the target variable Chance of Admit
CGPA is 0.882 correlated with the target variable Chance of Admit
Research is 0.546 correlated with the target variable Chance of Admit
Chance of Admit is 1.0 correlated with the target variable Chance of Admit


In [18]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['GRE Score','TOEFL Score', 'CGPA'],outputCol='features' )

In [19]:
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

###  Linear Regression Model

In [20]:
from pyspark.ml.regression import LinearRegression

final_data = output_data.select('features', 'Chance of Admit')

In [21]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [22]:
train, test = final_data.randomSplit([0.7,0.3])


In [23]:
models = LinearRegression(featuresCol= 'features', labelCol='Chance of Admit')

model= models.fit(train)


In [24]:
print('Coefficients:', model.coefficients)

print('Intercept:', model.intercept)

Coefficients: [0.002864948823869475,0.002804815749520415,0.14018218615399416]
Intercept: -1.6874308155482027


In [25]:
summary = model.summary

In [26]:
print('RMSE :', summary.rootMeanSquaredError)

print('r2 :', summary.r2)

RMSE : 0.06351018972558198
r2 : 0.8006778393705509


###  Evaluate & Save the Model

In [27]:
predictions= model.transform(test)

In [28]:
predictions.show()

+------------------+---------------+-------------------+
|          features|Chance of Admit|         prediction|
+------------------+---------------+-------------------+
|  [293.0,97.0,7.8]|           0.64| 0.5174873695501878|
|  [295.0,93.0,7.2]|           0.46|  0.427888692507449|
|  [296.0,97.0,7.8]|           0.49| 0.5260822160217966|
|[296.0,101.0,7.68]|            0.6| 0.5204796166813987|
| [297.0,96.0,7.89]|           0.43| 0.5387587458500049|
|  [298.0,98.0,7.5]|           0.44|0.49256227357285765|
|[298.0,101.0,7.86]|           0.54| 0.5514423078368567|
| [299.0,96.0,7.86]|           0.54| 0.5402831779131243|
|[299.0,100.0,7.42]|           0.42|0.48982227900344855|
|[299.0,100.0,7.88]|           0.68| 0.5543060846342858|
| [299.0,106.0,8.4]|           0.64| 0.6440297159314854|
| [300.0,95.0,8.22]|           0.62| 0.5908088980029111|
|[300.0,100.0,8.66]|           0.64| 0.6665131386582708|
|[300.0,102.0,7.87]|           0.56| 0.5613788430956559|
| [301.0,96.0,7.56]|           

In [29]:
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit', metricName='r2')

print('r2 score:', evaluator.evaluate(predictions))

r2 score: 0.8129945464385409


In [30]:
model.save('model')

In [31]:
from pyspark.ml.regression import LinearRegressionModel

model_new= LinearRegressionModel.load('model')

In [32]:
model_new

LinearRegressionModel: uid=LinearRegression_320ffd78e9e6, numFeatures=3